In [1]:
import numpy as np
import pandas as pd

In [3]:
# File with public supply total GW + SW
usgs_2020 = pd.read_csv(r'C:\Users\mhardika\Documents\AMO\2050\analysis_files_2024\PS_HUC12_GW_2000_2020.csv')

In [10]:
dpc_2020 = pd.read_csv(r'C:\Users\mhardika\Documents\AMO\2050\analysis_files_2024\delivery_water_use_model\predictions\national_dpc_predictions.csv')

In [13]:
dpc_2020.tail(500)

,wsa_agidf,year,population,lat,lon,dpc_0,dpc_1,dpc_2,dpc_3,dpc_4,...,dpc_90,dpc_91,dpc_92,dpc_93,dpc_94,dpc_95,dpc_96,dpc_97,dpc_98,dpc_99
394447,nv0000289,2004,15155.0,35.840671,-114.925784,157.399410,152.216500,166.76303,139.10843,115.963300,...,150.64640,144.004400,166.374220,151.491560,151.996720,172.324190,148.223250,150.380500,159.278800,158.476200
394448,nv0000289,2005,15157.0,35.840671,-114.925784,157.199250,152.906130,176.19308,144.44707,122.833500,...,150.69742,151.850430,164.778700,146.249970,152.562760,175.059810,145.726720,139.794220,156.303440,161.499480
394449,nv0000289,2006,15159.0,35.840671,-114.925784,153.784650,153.673100,168.90167,151.27391,122.526730,...,154.42017,157.216430,172.339050,152.585130,155.125560,170.587500,144.337130,140.040180,157.183980,159.259480
394450,nv0000289,2007,15161.0,35.840671,-114.925784,156.644380,155.219270,168.90167,150.14621,123.462234,...,154.42017,157.435360,177.844830,153.439500,153.786270,171.569500,151.642090,140.894270,159.977140,160.756010
394451,nv0000289,2008,15163.0,35.840671,-114.925784,155.739850,155.510790,168.90167,151.44308,122.978510,...,155.47733,158.458020,178.051740,153.648450,156.257540,171.027470,151.161180,141.350430,159.977140,161.464200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394942,utah26006,2016,13603.0,40.432212,-111.318557,77.609500,77.627010,79.10344,84.70235,82.545870,...,72.97945,80.691086,86.250656,83.256340,81.624140,82.876340,87.670160,88.975630,86.348420,88.785950
394943,utah26006,2017,14049.0,40.432212,-111.318557,74.325950,74.315710,79.23992,76.51398,79.449066,...,67.98091,77.260740,80.963930,78.396515,80.788240,76.651436,82.138200,81.578000,80.730450,81.231860
394944,utah26006,2018,14495.0,40.432212,-111.318557,75.140884,78.137856,79.54416,79.65482,82.894880,...,69.47569,77.581510,82.245790,81.036100,81.577680,84.307000,88.602010,81.379620,85.274704,82.470276
394945,utah26006,2019,14941.0,40.432212,-111.318557,75.580330,70.884224,76.18018,77.69269,79.271380,...,69.05887,75.582260,81.724420,74.263466,79.254166,78.417650,82.515625,84.069046,82.560104,80.868130


In [6]:
usgs_2020.head(20)

,Year,Month,010100020101,010100020102,010100020103,010100020104,010100020105,010100020201,010100020202,010100020203,...,181002041404,181002041501,181002041502,181002041503,181002041504,181002041505,181002041506,181002041507,181002041508,181002041600
0,2000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2000,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2000,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2000,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2000,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2000,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2000,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2000,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2000,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2000,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


15735.281619921552